In [3]:
import os
import argparse
import yaml
import glob
from tqdm import trange
import pandas as pd
import yaml
import torch
import scipy
import numpy as np
import argparse
import os
from glob import glob
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

In [4]:
from dataset import CTDataset 

In [5]:
from train import create_dataloader, load_model 

In [ ]:
from dataset import CTDataset 
import os
import argparse
import yaml
import glob
from tqdm import trange
import pandas as pd
import yaml
import torch
import scipy
import numpy as np
import argparse
import os
from glob import glob
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from train import create_dataloader, load_model 


In [ ]:
## linux path: /home/azureuser/snow-Dayz/configs/exp_resnet50_2classes.yaml
### local path 
cfg = yaml.safe_load(open('/Users/cmbreen/code/snow-Dayz/configs/exp_resnet50_2classes.yaml'))

In [ ]:
model, current_epoch = load_model(cfg, outdir)

In [ ]:


## We need the cfg (model configuration), model 
### could make this a class rather than a function
def predict(cfg, dataLoader, model):
    with torch.no_grad(): # no gradients needed for prediction
        predictions = []
        predict_labels = [] 
        labels = []
        ##### may need to adjust this in the dataloader for the sequence:
        for idx, (data, label) in enumerate(dataLoader): 
            prediction = model(data) 
            predict_label = torch.argmax(prediction, dim=1) 

        predictions.append(prediction)
        predict_labels.append(int(predict_label))
        labels.append(int(label))

    prediction_label = model.predict(image)
    label = max (prediction_label)

    #print(pred_mask.shape) ## probabilities

    return prediction